In [1]:
import numpy
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
cd ./American_dad_script/

C:\Users\berte\Desktop\M2_AIC\NLP\Projet\American_dad_script


In [4]:
full_text = []

for filename in os.listdir(os.getcwd()):
    raw_text = open(filename, 'r', encoding='utf-8').read()
    full_text.append(raw_text.lower())


In [5]:
flat_list = [item for sublist in full_text for item in sublist]

In [6]:
flat_list = flat_list[:75000]

In [7]:
# create mapping of unique chars to integers
chars = sorted(list(set(flat_list)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [8]:
for key, value in char_to_int.items() :
    print (key, value)


  0
! 1
" 2
$ 3
% 4
' 5
, 6
- 7
. 8
0 9
1 10
2 11
3 12
4 13
5 14
7 15
8 16
9 17
: 18
? 19
a 20
b 21
c 22
d 23
e 24
f 25
g 26
h 27
i 28
j 29
k 30
l 31
m 32
n 33
o 34
p 35
q 36
r 37
s 38
t 39
u 40
v 41
w 42
x 43
y 44
z 45
ä 46
é 47


In [9]:
n_chars = len(flat_list)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  75000
Total Vocab:  48


In [10]:
...
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = flat_list[i:i + seq_length]
	seq_out = flat_list[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  74900


In [11]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
...
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
...
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [14]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)


Epoch 1/20
74900/74900 [==============================] - 386s 5ms/step - loss: 3.0295

Epoch 00001: loss improved from inf to 3.02947, saving model to weights-improvement-01-3.0295.hdf5
Epoch 2/20
74900/74900 [==============================] - 376s 5ms/step - loss: 2.8682

Epoch 00002: loss improved from 3.02947 to 2.86823, saving model to weights-improvement-02-2.8682.hdf5
Epoch 3/20
74900/74900 [==============================] - 384s 5ms/step - loss: 2.7918

Epoch 00003: loss improved from 2.86823 to 2.79176, saving model to weights-improvement-03-2.7918.hdf5
Epoch 4/20
74900/74900 [==============================] - 354s 5ms/step - loss: 2.7510

Epoch 00004: loss improved from 2.79176 to 2.75103, saving model to weights-improvement-04-2.7510.hdf5
Epoch 5/20
74900/74900 [==============================] - 398s 5ms/step - loss: 2.7181

Epoch 00005: loss improved from 2.75103 to 2.71806, saving model to weights-improvement-05-2.7181.hdf5
Epoch 6/20
74900/74900 [========================

KeyboardInterrupt: 

In [15]:
import sys

In [16]:
...
# load the network weights
filename = "weights-improvement-01-2.9576.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

OSError: Unable to open file (unable to open file: name = 'weights-improvement-01-2.9576.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [105]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [108]:
...
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" . l'm just gonna mop it up with ''exodus''. - hello? - hayley? roger. got a sec? roger? whoa! ls thi "
  -  oo  -                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               